In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable


def cambiar_(x):
    x_list = x.split('-')
    
    return f'{x_list[0]}_{x_list[1]}'

#Seteamos para poder desplegar todas las columnas
pd.set_option("display.max_columns", None)

#Importamos la base de datos previamente depurada por el equipo
datos_calibracion = pd.read_excel('data_calibracion.xlsx')
datos_productos = pd.read_excel('productos_modelo.xlsx')
datos_productos.Tipo = [x.rstrip() for x in datos_productos.Tipo]
datos_productos.Nombre = [cambiar_(x) for x in datos_productos.Nombre]
datos_productos_1 = datos_productos.loc[:, [x for x in datos_productos.columns if x[0:3] == 'Num']]

#Definimos dataframes que los vamos a utilizar como diccionarios
dic_prod = datos_productos.to_dict('records')
diccionario_atributos = dict()
for diccionario in dic_prod:
    alt = diccionario['Numero_Alternativa']
    diccionario_atributos[alt] = diccionario

In [2]:
def columna_estaciones(x):
    x_ = x.split('_')
    x_1 = int(x_[0])
    if x_1 >= 3 and x_1 <= 5:
        return 'Otoño'
    elif x_1 >= 6 and x_1 <= 8:
        return 'Invierno'
    elif x_1 >= 9 and x_1 <= 11:
        return 'Primavera'
    else:
        return 'Verano'


diccionario_estaciones = {'Verano': 1,
                         'Otoño': 2,
                          'Invierno':3,
                         'Primavera': 4}
datos_calibracion.insert(2, 'Estacion', [diccionario_estaciones[columna_estaciones(x)] for x in datos_calibracion.Mes_Cliente])

In [3]:
datos_calibracion = datos_calibracion.iloc[:, 2:]
datos_calibracion.Costo1.unique()

array([    0.   , 30553.488, 29357.566, 32987.165, 34336.648, 33484.097])

In [4]:
#Transformamos la Data Frame, en el database

database = db.Database("DatosCalibracion", datos_calibracion)
productos_db = db.Database("DatosProductos", datos_productos_1)
globals().update(database.variables)
globals().update(productos_db.variables)

In [5]:
def buscar(x, nombre):
    dic_ = diccionario_atributos[x]
    #print(dic_)
    return dic_[nombre]

In [6]:
lis_str = [buscar(i, 'Nombre') for i in range(1, 14)]

In [7]:
#Definimos las constantes entre alternativas, fijamos la constante modal del bus (se debe fijar una constante modal)

ASC_0 = Beta(f'ASC_No_Compra', 0, None, None, 1)
ASC_1 = Beta(f'ASC_{lis_str[0]}', 0, None, None, 0)
ASC_2 = Beta(f'ASC_{lis_str[1]}', 0, None, None, 0)
ASC_3 = Beta(f'ASC_{lis_str[2]}', 0, None, None, 0)
ASC_4 = Beta(f'ASC_{lis_str[3]}', 0, None, None, 0)
ASC_5 = Beta(f'ASC_{lis_str[4]}', 0, None, None, 0)
ASC_6 = Beta(f'ASC_{lis_str[5]}', 0, None, None, 0)
ASC_7 = Beta(f'ASC_{lis_str[6]}', 0, None, None, 0)
ASC_8 = Beta(f'ASC_{lis_str[7]}', 0, None, None, 0)
ASC_9 = Beta(f'ASC_{lis_str[8]}', 0, None, None, 0)
ASC_10 = Beta(f'ASC_{lis_str[9]}', 0, None, None, 0)
ASC_11 = Beta(f'ASC_{lis_str[10]}', 0, None, None, 0)
ASC_12 = Beta(f'ASC_{lis_str[11]}', 0, None, None, 0)
ASC_13 = Beta(f'ASC_{lis_str[12]}', 0, None, None, 0)


#Guardamos los datos de las constantes en una lista
constantes_modelo = [ASC_0, ASC_1, ASC_2, ASC_3, ASC_4, 
                     ASC_5, ASC_6, ASC_7, ASC_8, ASC_9,
                    ASC_10, ASC_11, ASC_12, ASC_13]

Precio1_SCALED = DefineVariable('Precio1_SCALED', Costo1 ,database)
Precio2_SCALED = DefineVariable('Precio2_SCALED', Costo2 ,database)
Precio3_SCALED = DefineVariable('Precio3_SCALED', Costo3 ,database)
Precio4_SCALED = DefineVariable('Precio4_SCALED', Costo4 ,database)
Precio5_SCALED = DefineVariable('Precio5_SCALED', Costo5 ,database)
Precio6_SCALED = DefineVariable('Precio6_SCALED', Costo6 ,database)
Precio7_SCALED = DefineVariable('Precio7_SCALED', Costo7 ,database)
Precio8_SCALED = DefineVariable('Precio8_SCALED', Costo8 ,database)
Precio9_SCALED = DefineVariable('Precio10_SCALED', Costo10 ,database)
Precio10_SCALED = DefineVariable('Precio9_SCALED', Costo9 ,database)
Precio11_SCALED = DefineVariable('Precio11_SCALED', Costo11 ,database)
Precio12_SCALED = DefineVariable('Precio12_SCALED', Costo12 ,database)
Precio13_SCALED = DefineVariable('Precio13_SCALED', Costo13 ,database)


#Definimos los beta
B_precio = Beta('B_precio', 0, None, None, 0)

#En vez de poner un beta de materialidad, pondremos un beta de reciclabilidad
#B_materialidad = Beta('B_materialidad', 0, None, None, 0)
#B_aplicacion = Beta('B_aplicacion', 0, None, None, 0)
B_tipo = Beta('B_tipo', 0, None, None, 0)

Lamb_Film = Beta('Lamb_Film', 1.05, None, None, 0)
Lamb_Papel = Beta('Lamb_Papel', 1.05, None, None, 0)
#Lamb_Pad = Beta('Lamb_Pad', 1.05, None, None, 0)

#Definimos los mercados de los clientes
Carnico = Mercado == 1
Marino = Mercado == 2
Retail = Mercado == 3

#Definimos los estimadores de los mercados
B_carnico = Beta('B_carnico', 0, None, None, 0)
B_marino = Beta('B_marino', 0, None, None, 0)
B_retail = Beta('B_retail', 0, None, None, 0)

#Estimamos el beta de la reciclabilidad
B_reciclabilidad = Beta('B_reciclabilidad', 0, None, None, 0)

#Definimos las estaciones del año como una variable explicativa

Verano = Estacion == 1
#Otono = Estacion == 2
#Invierno = Estacion == 3


B_Verano = Beta('B_Verano', 0, None, None, 0) 
#B_Otono = Beta('B_Otono', 0, None, None, 0) 
#B_Invierno = Beta('B_Invierno', 0, None, None, 0)

 



In [8]:

#Defimos las utilidades, ahora vamos a incorporar las estaciones del año
#para ver el efecto que nos provoca en las distintas variaciones

V0 = ASC_0

#Utilidad alternativa 1
V1 = ASC_1 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio1_SCALED  \
    + B_reciclabilidad*buscar(1, 'Reciclabilidad') \
    + buscar(1, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
    

#Utilidad alternativa 2
V2 = ASC_2 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio2_SCALED  \
    + B_reciclabilidad*buscar(2, 'Reciclabilidad') \
    + buscar(2, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
     


#Utilidad alternativa 3
V3 = ASC_3 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio3_SCALED \
        + B_reciclabilidad*buscar(3, 'Reciclabilidad')\
    + buscar(3, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
    

#Utilidad alternativa 4
V4 = ASC_4 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio4_SCALED \
        + B_reciclabilidad*buscar(4, 'Reciclabilidad') \
    + buscar(4, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )


#Utilidad alternativa 5
V5 = ASC_5 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio5_SCALED \
        + B_reciclabilidad*buscar(5, 'Reciclabilidad') \
    + buscar(5, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        


#Utilidad alternativa 6
V6 = ASC_6 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio6_SCALED \
        + B_reciclabilidad*buscar(6, 'Reciclabilidad') \
    + buscar(6, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        


#Utilidad alternativa 7
V7 = ASC_7 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio7_SCALED \
        + B_reciclabilidad*buscar(7, 'Reciclabilidad') \
    + buscar(7, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        


#Utilidad alternativa 8
V8 = ASC_8 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio8_SCALED \
        + B_reciclabilidad*buscar(8, 'Reciclabilidad') \
    + buscar(8, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        


#Utilidad alternativa 9
V9 = ASC_9 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio9_SCALED \
        + B_reciclabilidad*buscar(9, 'Reciclabilidad') \
    + buscar(9, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        


#Utilidad alternativa 10
V10 = ASC_10 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio10_SCALED \
        + B_reciclabilidad*buscar(10, 'Reciclabilidad') \
    + buscar(10, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        
        

#Utilidad alternativa 11
V11 = ASC_11 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio11_SCALED \
        + B_reciclabilidad*buscar(11, 'Reciclabilidad') \
    + buscar(11, 'Numero_Tipo')*(B_tipo + B_Verano*Verano  )
        
        

#Utilidad alternativa 12
V12 = ASC_12 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio12_SCALED \
        + B_reciclabilidad*buscar(12, 'Reciclabilidad') \
    + buscar(12, 'Reciclabilidad')*(B_reciclabilidad + B_Verano*Verano )
        
        

#Utilidad alternativa 13
V13 = ASC_13 + (B_precio + B_carnico*Carnico +B_retail*Retail + B_marino*Marino)*Precio13_SCALED \
    + B_reciclabilidad*buscar(13, 'Reciclabilidad') \
    + buscar(13, 'Reciclabilidad')*(B_reciclabilidad + B_Verano*Verano  )
        
        

In [9]:
#Asociamos las utilidades con las distintas alternativas

V = {0: V0,
     1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7,
     8: V8,
     9: V9,
     10: V10,
     11: V11,
     12: V12,
     13: V13}


# Asociamos la condicion de disponibilidad de las alternativas (no necesariamente todas las alternativas estan disponibles para todos los individuos)

av = {0: Disp0,
      1: Disp1,
      2: Disp2,
      3: Disp3,
      4: Disp4,
      5: Disp5,
      6: Disp6, 
      7: Disp7,
      8: Disp8,
      9: Disp9,
      10: Disp10,
      11: Disp11,
      12: Disp12,
      13: Disp13}


In [11]:
#Buscamos las alternativas por nideos
from collections import defaultdict
dic_nidos = defaultdict(list)
dic_nidos['Resto'].append(0)
for key, value in diccionario_atributos.items():
    nombre_linea = value['Linea']
    if nombre_linea == 'Zuncho' or nombre_linea == 'Paños Absorbentes':
    #if nombre_linea == 'Zuncho':
       # print(nombre_linea)
        dic_nidos['Resto'].append(key)
    else:
        dic_nidos[nombre_linea].append(key)

#Creamos los nidos
Film = Lamb_Film, dic_nidos['Film']
Papel = Lamb_Papel, dic_nidos['Papel Cerafinado']
#Pad = Lamb_Pad, dic_nidos['Paños Absorbentes']
Resto = 1.0, dic_nidos['Resto']

nidos = Film, Papel, Resto

# Probabilidad
logprob = models.lognested(V,av,nidos,PRODUCTO)
    

In [12]:
models.checkValidityNestedLogit(V, nidos)

(True, 'The nested logit model is based on a partition. ')

In [13]:
import biogeme.messaging as msg
logger = msg.bioMessage()
#logger.setDebug()
#logger.setWarning()
logger.setSilent()
#logger.setDetailed()

In [14]:
import os
#Modelo
biogeme = bio.BIOGEME(database, logprob)
nombre = 'HL_Variacion_sistematica_recilabilidad_estaciones_9'
biogeme.modelName = nombre

#Estimacion 
resultados = biogeme.estimate()
biogeme.saveIterations = False
biogeme.generatePickle = False
os.remove(f'{nombre}.pickle')
os.remove(f'__{nombre}.iter')
os.remove(f"{nombre}.html")
resultados.writeLaTeX()

In [21]:
import scipy.stats as st

estadisticos = resultados.getEstimatedParameters()
nests = [x for x in estadisticos.index if x[0:3] == 'Lam']
print(nests)
nivel_significancia = 0.1
for n in nests:
    bool_ = abs((estadisticos.Value[n] -1)/estadisticos['Rob. Std err'][n]) >= st.distributions.norm.ppf(1 - nivel_significancia/2)
    if bool_ == True:
        print(f'Nido {n} no colapsa')
    else:
        print(f'Nido {n} colapsa')

['Lamb_Film', 'Lamb_Pad', 'Lamb_Papel']
Nido Lamb_Film colapsa
Nido Lamb_Pad colapsa
Nido Lamb_Papel no colapsa


In [27]:
def modelo_solo_constantes(constantes_modelo, database, numero):
    
    datos_calibracion = pd.read_excel('data_calibracion.xlsx')
    database_ = db.Database("DatosCalibracion", datos_calibracion)

    globals().update(database_.variables)
    V = dict()
    
    contador = 0
    
    for i in constantes_modelo:
        
        V[contador] = i
        contador += 1
        

    av = {0: Disp0,
          1: Disp1,
          2: Disp2,
          3: Disp3,
          4: Disp4,
          5: Disp5,
          6: Disp6, 
          7: Disp7,
          8: Disp8,
          9: Disp9,
          10: Disp10,
          11: Disp11,
          12: Disp12,
          13: Disp13}
        
        
    #Determinamos la probabilidad de ocurencia (log verosimilitud)
    logprob = models.loglogit(V, av, PRODUCTO)
    
    import biogeme.messaging as msg
    logger = msg.bioMessage()
    #logger.setDebug()
    #logger.setWarning()
    logger.setSilent()
    #logger.setDetailed()
    
    
    #Modelo
    biogeme = bio.BIOGEME(database, logprob)
    nombre = f"MNL_Constantes{numero}"
    biogeme.modelName = nombre
    
    resultados_ = biogeme.estimate()
    biogeme.saveIterations = False
    biogeme.generatePickle = False
    os.remove(f'{nombre}.pickle')
    os.remove(f'__{nombre}.iter')
    os.remove(f'{nombre}.html')
    
    estadisticos = resultados_.getGeneralStatistics()
        
        
    return resultados_, estadisticos

#Sacamos la informacion de la imagen que muestra en el HTML
def sacar_estadisticas_resultado(resultados, constantes_modelo, database, numero):
    
    #Para el analisis estimamos el modelo de solo constantes, y determinamos su verosimilitud
    MNL_SC_resutado, estadisticos = modelo_solo_constantes(constantes_modelo, database, numero)
    
    estadisticas_resultado = resultados.getGeneralStatistics()
    df_estimadores_estadisticos = resultados.getEstimatedParameters()
    breve_resumen_modelo = resultados.shortSummary()
    parametros = resultados.getBetaValues()
    
    #Calculo de indicadores
    rho_cuadrado = 1 - (estadisticas_resultado['Final log likelihood'][0]/estadisticas_resultado['Init log likelihood'][0])
    rho_barra_bio = estadisticas_resultado['Rho-square-bar for the init. model'][0]
    AIC = estadisticas_resultado['Akaike Information Criterion']
    BIC = estadisticas_resultado['Bayesian Information Criterion']
    rho_ajustado_constantes = 1 - (estadisticos['Final log likelihood'][0]/estadisticos['Init log likelihood'][0])
    rho_barra = 1 - (estadisticas_resultado['Final log likelihood'][0]/estadisticos['Final log likelihood'][0]) 
    rho_constantes = 1 - (estadisticos['Final log likelihood'][0]/estadisticos['Init log likelihood'][0])
    
    #Diccionario de indicadores
    indicadores_significancia = { 'rho_cuadrado': rho_cuadrado,
                                  'rho_barra_bio': rho_barra_bio,
                                  'rho_constantes': rho_constantes,
                                  'rho_ajustado': rho_barra, 
                                  'AIC': AIC,
                                  'BIC': BIC }
    
    
    return estadisticas_resultado, df_estimadores_estadisticos, breve_resumen_modelo, parametros, indicadores_significancia

In [28]:
estadisticas_resultado, df_estimadores_estadisticos, breve_resumen_modelo, parametros, indicadores = sacar_estadisticas_resultado(resultados,
                                                                                                                                  constantes_modelo,
                                                                                                                                   database,3)
                                                                                                                               

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/biogeme/algorithms.py:342: RuntimeWarning: overflow encountered in double_scalars
  else (self.lowerBounds[i] - x[i]) / d[i]
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/biogeme/algorithms.py:340: RuntimeWarning: overflow encountered in double_scalars
  (self.upperBounds[i] - x[i]) / d[i]


In [36]:
indicadores

{'rho_cuadrado': 0.37973467316063436,
 'rho_barra_bio': 0.3688525040947358,
 'rho_constantes': 0.374365834210711,
 'rho_ajustado': 0.0068126525869876,
 'AIC': (2667.9226021788304, '.7g'),
 'BIC': (2798.841208989526, '.7g')}